<a href="https://colab.research.google.com/github/munguu0613/47474793-COMP8240-Individual-Dataset-20NewsGroup/blob/main/COMP8240_Group_A_CLSTM_47474793.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import layers

In [ ]:
# !pip install datasets

In [ ]:
# !pip install tensorflow scikit-learn datasets


BINARY CLASSIFICATION C-LSTM on 20 NEWSGROUP

In [ ]:
# !pip install tensorflow scikit-learn datasets


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups

# Step 1: Load and preprocess the 20 Newsgroups dataset
# Select two categories for binary classification, e.g., 'sci.med' and 'rec.autos'
categories = ['sci.med', 'rec.autos']
newsgroups_data = fetch_20newsgroups(subset='all', categories=categories, remove=('headers', 'footers', 'quotes'))

# Text data and binary labels (0 or 1 based on category index)
texts = newsgroups_data.data
labels = newsgroups_data.target  # 0 for 'sci.med', 1 for 'rec.autos'

# Tokenization and padding
VOCAB_SIZE = 10000
MAX_LEN = 300

tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
x_data = pad_sequences(sequences, maxlen=MAX_LEN)
y_data = np.array(labels)

# Split data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

# Step 2: Load GloVe embeddings
def load_glove_embeddings(glove_file_path, embedding_dim=300):
    embeddings_index = {}
    with open(glove_file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            embedding_vector = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = embedding_vector
    return embeddings_index

# Create embedding matrix
def create_embedding_matrix(word_index, glove_embeddings, vocab_size, embedding_dim=300):
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, i in word_index.items():
        if i < vocab_size:
            embedding_vector = glove_embeddings.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

# Load pre-trained GloVe embeddings (update with your actual path)
glove_file_path = "C:\Users\mungu\OneDrive\Documents\MQ Units\COMP8240 - Applications of Data Science\Project Update\glove.6B.300d.txt"
glove_embeddings = load_glove_embeddings(glove_file_path)
embedding_matrix = create_embedding_matrix(tokenizer.word_index, glove_embeddings, VOCAB_SIZE)

# Step 3: Define the C-LSTM Model for Binary Classification
class CLSTMBinaryClassifier(tf.keras.Model):
    def __init__(self, vocab_size, embedding_matrix, max_length, num_filters=150, lstm_units=150, embedding_dim=300, dropout_rate=0.5, l2_reg_lambda=0.001):
        super(CLSTMBinaryClassifier, self).__init__()
        self.embedding = layers.Embedding(input_dim=vocab_size,
                                          output_dim=embedding_dim,
                                          input_length=max_length,
                                          weights=[embedding_matrix],
                                          trainable=True)
        self.embedding_dropout = layers.Dropout(rate=dropout_rate)

        # Convolutional layer with filter size 3
        self.conv_layer = layers.Conv2D(filters=num_filters,
                                        kernel_size=(3, embedding_dim),
                                        activation='relu', padding='valid')
        self.batch_norm = layers.BatchNormalization()

        # LSTM layer to capture dependencies
        self.lstm = layers.LSTM(lstm_units, return_sequences=False)
        self.dropout = layers.Dropout(rate=dropout_rate)

        # Output layer for binary classification
        self.fc = layers.Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.L2(l2_reg_lambda))

    def call(self, inputs, training=False):
        x = self.embedding(inputs)
        x = self.embedding_dropout(x, training=training)
        x = tf.expand_dims(x, -1)

        # Apply convolutional layer and batch normalization
        conv_out = self.conv_layer(x)
        conv_out = self.batch_norm(conv_out, training=training)
        conv_out = tf.squeeze(conv_out, 2)

        # Pass through LSTM
        rnn_outputs = self.lstm(conv_out)
        rnn_outputs = self.dropout(rnn_outputs, training=training)

        # Output for binary classification
        binary_output = self.fc(rnn_outputs)
        return binary_output

# Step 4: Initialize and compile the model
model = CLSTMBinaryClassifier(vocab_size=VOCAB_SIZE,
                              embedding_matrix=embedding_matrix,
                              max_length=MAX_LEN)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# Step 5: Train the model
history = model.fit(
    x_train, y_train,
    batch_size=64,
    epochs=10,
    validation_data=(x_test, y_test),
    verbose=1
)

# Step 6: Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_acc}')


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-6-899f5ee4db5d>, line 53)

C-LSTM FINE-GRAINED CLASSIFICATTION ON THE 20 NEWS GROUP DATASET

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups

# Step 1: Load and preprocess the 20 Newsgroups dataset
newsgroups_data = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

# Text data and fine-grained labels (20 categories)
texts = newsgroups_data.data
labels = newsgroups_data.target

# Tokenization and padding
VOCAB_SIZE = 10000
MAX_LEN = 300

tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
x_data = pad_sequences(sequences, maxlen=MAX_LEN)
y_data = np.array(labels)

# Split data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

# Step 2: Load GloVe embeddings
def load_glove_embeddings(glove_file_path, embedding_dim=300):
    embeddings_index = {}
    with open(glove_file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            embedding_vector = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = embedding_vector
    return embeddings_index

# Create embedding matrix
def create_embedding_matrix(word_index, glove_embeddings, vocab_size, embedding_dim=300):
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, i in word_index.items():
        if i < vocab_size:
            embedding_vector = glove_embeddings.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

# Load pre-trained GloVe embeddings (update this path)
glove_file_path = "/content/drive/MyDrive/glove/glove.6B.300d.txt"
glove_embeddings = load_glove_embeddings(glove_file_path)
embedding_matrix = create_embedding_matrix(tokenizer.word_index, glove_embeddings, VOCAB_SIZE)

# Step 3: Define the C-LSTM Model for Fine-Grained Classification (20 classes)
class CLSTMMultiClassClassifier(tf.keras.Model):
    def __init__(self, vocab_size, embedding_matrix, max_length, num_filters=150, lstm_units=150, num_classes=20, embedding_dim=300, dropout_rate=0.5, l2_reg_lambda=0.001):
        super(CLSTMMultiClassClassifier, self).__init__()
        self.embedding = layers.Embedding(input_dim=vocab_size,
                                          output_dim=embedding_dim,
                                          input_length=max_length,
                                          weights=[embedding_matrix],
                                          trainable=True)
        self.embedding_dropout = layers.Dropout(rate=dropout_rate)

        # Convolutional layer with filter size 3
        self.conv_layer = layers.Conv2D(filters=num_filters,
                                        kernel_size=(3, embedding_dim),
                                        activation='relu', padding='valid')
        self.batch_norm = layers.BatchNormalization()

        # LSTM layer to capture dependencies
        self.lstm = layers.LSTM(lstm_units, return_sequences=False)
        self.dropout = layers.Dropout(rate=dropout_rate)

        # Output layer for multi-class classification
        self.fc = layers.Dense(num_classes, activation='softmax', kernel_regularizer=tf.keras.regularizers.L2(l2_reg_lambda))

    def call(self, inputs, training=False):
        x = self.embedding(inputs)
        x = self.embedding_dropout(x, training=training)
        x = tf.expand_dims(x, -1)

        # Apply convolutional layer and batch normalization
        conv_out = self.conv_layer(x)
        conv_out = self.batch_norm(conv_out, training=training)
        conv_out = tf.squeeze(conv_out, 2)

        # Pass through LSTM
        rnn_outputs = self.lstm(conv_out)
        rnn_outputs = self.dropout(rnn_outputs, training=training)

        # Output for multi-class classification
        multi_class_output = self.fc(rnn_outputs)
        return multi_class_output

# Step 4: Initialize and compile the model
model = CLSTMMultiClassClassifier(vocab_size=VOCAB_SIZE,
                                  embedding_matrix=embedding_matrix,
                                  max_length=MAX_LEN,
                                  num_classes=20)

model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 5: Train the model
history = model.fit(
    x_train, y_train,
    batch_size=64,
    epochs=10,
    validation_data=(x_test, y_test),
    verbose=1
)

# Step 6: Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_acc}')


Epoch 1/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 12s 42ms/step - accuracy: 0.1911 - loss: 2.6503 - val_accuracy: 0.3586 - val_loss: 2.0495
Epoch 2/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.4776 - loss: 1.6343 - val_accuracy: 0.5549 - val_loss: 1.4209
Epoch 3/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.5918 - loss: 1.3165 - val_accuracy: 0.6085 - val_loss: 1.2747
Epoch 4/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.6366 - loss: 1.1911 - val_accuracy: 0.6244 - val_loss: 1.2685
Epoch 5/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.6676 - loss: 1.1052 - val_accuracy: 0.6610 - val_loss: 1.1554
Epoch 6/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.6812 - loss: 1.0501 - val_accuracy: 0.6634 - val_loss: 1.1286
Epoch 7/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.7108 - loss: 0.9676 - val_accuracy: 0.6658 - val_loss: 1.1243
Epoch 8/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.7269 - loss: 0.9116 - val_acc